In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from __future__ import print_function
import seaborn; seaborn.set()
from bot import sendMessage
from copy import copy

In [46]:
r_train = pd.read_csv('data/r_train.csv', index_col=0)
r_test = pd.read_csv('data/r_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [24]:
r_train = pd.read_csv('data/c_train.csv', index_col=0)
r_test = pd.read_csv('data/c_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [28]:
r_train = pd.read_csv('data/c_train80.csv', index_col=0)
r_test = pd.read_csv('data/c_train20.csv', index_col=0)
r_target = pd.read_csv('data/target80.csv', index_col=0)['0']
r_test_target = pd.read_csv('data/target20.csv', index_col=0)['0']

In [113]:
r_train = pd.read_csv('data/cc_train.csv', index_col=0)
r_test = pd.read_csv('data/cc_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [3]:
r_train = pd.read_csv('data/stm_train.csv', index_col=0)
r_test = pd.read_csv('data/stm_test.csv', index_col=0)
r_target = pd.read_csv('data/target.csv', index_col=0)['0']

In [3]:
r_train = pd.read_csv('data/stm_train80.csv', index_col=0)
r_test = pd.read_csv('data/stm_train20.csv', index_col=0)
r_target = pd.read_csv('data/target80.csv', index_col=0)['0']
r_test_target = pd.read_csv('data/target20.csv', index_col=0)['0']

In [29]:
train, test, target = r_train, r_test, r_target
print(train.shape, test.shape, target.shape)

(60815, 292) (15205, 292) (60815L,)


In [30]:
print(target.mean(), r_test_target.mean())

0.0395626079092 0.0395922393949


In [5]:
import os
mingw_path = 'C:\\Program Files\\mingw-w64\\x86_64-5.3.0-posix-seh-rt_v4-rev0\\mingw64\\bin'
os.environ['PATH'] = mingw_path + ';' + os.environ['PATH']

import xgboost as xgb

In [19]:
import gc; gc.collect()

14

In [31]:
dtrain = xgb.DMatrix(train, target)
dtest = xgb.DMatrix(test)

In [13]:
params_ho = {'alpha': .6,
          'colsample_bytree': .68,
          'eta': .03,
          'eval_metric': 'auc',
          'lambda': 1.,
          'max_depth': 3,
          'min_child_weight': 2.0,
          'objective': 'binary:logistic',
          'seed': 0,
          'silent': 1,
          'subsample': .55}

params_r = {'objective': 'binary:logistic', 
          'eval_metric': 'auc',
          'booster': 'gbtree',
          'eta': 0.0202048,
          'max_depth': 5,
          'subsample': 0.6815,
          'colsample_bytree': 0.701,
          'silent': 1,
          'seed': 0
}

In [14]:
params = params_r

In [32]:
res = []
for x in range(1):
    xcv = xgb.cv(params, dtrain, num_boost_round=560, 
           nfold=5, stratified=True, verbose_eval=True, 
           early_stopping_rounds=20, seed=x + 10);
    res.append((xcv.iloc[-1].name, xcv.iloc[-1]['test-auc-mean']))
    sendMessage('done %d %f' % (x, res[-1][1]))
map(print, res);
print(np.array(map(lambda x: x[1], res)).mean());

Will train until cv error hasn't decreased in 20 rounds.
[0]	cv-test-auc:0.4999988+0.000362306720887	cv-train-auc:0.502817+0.000381488138741
[1]	cv-test-auc:0.5003332+0.000944042032963	cv-train-auc:0.5043824+0.000329059629855
[2]	cv-test-auc:0.5009716+0.000521744995184	cv-train-auc:0.505861+0.00124838535717
[3]	cv-test-auc:0.5004036+0.000607501967075	cv-train-auc:0.5074092+0.00144907286221
[4]	cv-test-auc:0.5003708+0.000925285123624	cv-train-auc:0.5079942+0.00192096854737
[5]	cv-test-auc:0.5005312+0.00100741359927	cv-train-auc:0.5084206+0.00183111753855
[6]	cv-test-auc:0.5007078+0.00110456876653	cv-train-auc:0.508966+0.00142679627137
[7]	cv-test-auc:0.5006048+0.00109788804529	cv-train-auc:0.5090852+0.00143900179291


KeyboardInterrupt: 

In [130]:
map(print, res);

(559, 0.8441126000000001)
(559, 0.84463880000000002)
(559, 0.84312259999999983)
(500, 0.84136919999999993)


In [96]:
print('cv10 c_data r_ params', 0.8413659)
print('cv10 c+_data r_params', 0.84130432)

cv10 c_data r_ params 0.8413659


In [28]:
from time import time
res = []
for x in range(1):
    start = time()
    params['seed'] = x
    gbm = xgb.train(params, dtrain, num_boost_round=800)
    pred = gbm.predict(dtest)
    res.append(pred)
    sendMessage('done %d %f min' % (x, (time() - start) / 60.))

Exception AttributeError: "'Booster' object has no attribute 'handle'" in <bound method Booster.__del__ of <xgboost.core.Booster object at 0x0000000020B37E48>> ignored


KeyboardInterrupt: 

In [23]:
preds = np.array(res[:]).mean(axis=0)

In [24]:
preds.shape

(75818L,)

In [25]:
sample = pd.read_csv('data/sample_submission.csv', index_col='ID')
sample.TARGET = preds
sample.to_csv('submission/xgb_stm_800nr_bag10.csv', index_label='ID')

In [27]:
sendMessage('done')

In [30]:
from kaggle_submit import submit
submit('mike0sv', 'morr238', 'bnp-paribas-cardif-claims-management', './data/sample_submission.csv', 'scrip-test')

FormNotFoundError: no form matching nr 0

In [9]:
from time import time
res = []
dtest = xgb.DMatrix(test, label=r_test_target)
for x in range(1):
    start = time()
    params['seed'] = x
    cv_res = dict()
    ev = [(dtest, 'holdout')]
    gbm = xgb.train(params, dtrain, num_boost_round=1000, maximize=True,
                    evals=ev, evals_result=cv_res, verbose_eval=True, early_stopping_rounds=20)
    #xgb.cv(params, dtrain, num_boost_round=560, 
    #       nfold=5, stratified=True, verbose_eval=False, 
     #      early_stopping_rounds=20, seed=x + 10)
    sendMessage('done %d %f min %s' % (x, (time() - start) / 60., str(cv_res['holdout']['auc'][-1])))

Will train until holdout error hasn't decreased in 20 rounds.
[0]	holdout-auc:0.499760
[1]	holdout-auc:0.500386
[2]	holdout-auc:0.500351
[3]	holdout-auc:0.500180
[4]	holdout-auc:0.499907
[5]	holdout-auc:0.499565
[6]	holdout-auc:0.499565
[7]	holdout-auc:0.501155
[8]	holdout-auc:0.501641
[9]	holdout-auc:0.501641
[10]	holdout-auc:0.501640
[11]	holdout-auc:0.502298
[12]	holdout-auc:0.502298
[13]	holdout-auc:0.502299
[14]	holdout-auc:0.501755
[15]	holdout-auc:0.501835
[16]	holdout-auc:0.502532
[17]	holdout-auc:0.502498
[18]	holdout-auc:0.502497
[19]	holdout-auc:0.503868
[20]	holdout-auc:0.503765
[21]	holdout-auc:0.503766
[22]	holdout-auc:0.503659
[23]	holdout-auc:0.503660
[24]	holdout-auc:0.503660
[25]	holdout-auc:0.503660
[26]	holdout-auc:0.503354
[27]	holdout-auc:0.503354
[28]	holdout-auc:0.502885
[29]	holdout-auc:0.502879
[30]	holdout-auc:0.502879
[31]	holdout-auc:0.502879
[32]	holdout-auc:0.502879
[33]	holdout-auc:0.503708
[34]	holdout-auc:0.503705
[35]	holdout-auc:0.503508
[36]	holdout